Itay Koren

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection
import psycopg2

In [3]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-sales-data-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
inspector = reflection.Inspector.from_engine(engine)


<ipython-input-3-60b087478d5e>:14: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = reflection.Inspector.from_engine(engine)


In [5]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

rep_sales table:

        OrderDate: date when the order was placed

        Region: geographical area in which the sale was made.

        Rep: sales representative's name

        Item: name of the item sold

        Units: number of units sold

        UnitCost: cost of one unit

        Total: total cost of the order - Units x UnitCost

Item table:

        Item: name of the item

        MinPrice: minimum price that the item can be sold.

1. What region had the most units sold for pencil?  


In [12]:
query = '''
select 
    "Region",
    sum("Units")
    
from 
    rep_sales
where
    "Item" = 'Pencil'
group by
    "Region"
order by
    sum desc
limit 1;
'''

queryResult(query)

,Region,sum
0,Central,498



2. For each sale above the minimum price the rep gets 10% commission of the total price as a bonus. How many transactions weren’t calculated for commissions?


In [20]:
query = '''
select 
    count("Rep")
from 
    rep_sales
    left join item
    on item."Item" = rep_sales."Item"
where 
    "MinPrice" >= "Unit_Cost"

'''

queryResult(query)

,count
0,30



3. For West region what was the share of each sales rep in the data?


In [51]:
query = '''
select
    "Rep",
    100 * rep_total_sales / sum(rep_total_sales) over() as rep_share
from
    (
    select 
        "Rep",
        sum("Units" * "Unit_Cost") as rep_total_sales
    from 
        rep_sales
    where 
        "Region" = 'West'
    group by 
        "Rep"
    ) as subq
group by 
    "Rep", rep_total_sales
'''

queryResult(query)

,Rep,rep_share
0,Sorvino,38.585115
1,Thompson,61.414885



4. For the item Binder what was the share of each sales rep in the data?



5. For Jardine what was the share of each item sold in the data?



6. Which item is the most profitable to sell? Get the percentage of the diff from unit price to minimum and rank it for the entire data



7. Which sales rep had the highest price difference from the minimum price on these items combined: pen, pen set and pencil?